## Event Classification from EEG
#### Tyrome Sweet and Taran Rallings

### Introduction


The following analyes EEG data taken in experiments where participants where exposed to light and sound events. This code cleans that 32 channel EEG data and uses a long short-term memory recurrent neural network to classify the time following light or sound events by which event occured. 

### Data Prep



In [1]:
# setting the random seed for reproducibility
import random
seed=42
random.seed(seed)

In [2]:
# import libraries 
import itertools
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

### Load the Data

In [3]:
# eeg1 and events1 are the test data from a single person
# code assumes eeg1 and events1 are csv files in the current working directory

eeg1 = pd.read_csv("full_data_shuffle_24_04(2997).csv", delimiter="\t")
new_columns = eeg1.columns.values 
new_columns[0] = 'time'     
new_columns[33] = 'sample' 
eeg1.columns = new_columns

events1 = pd.read_csv("events1.csv") #, delimiter="\t"


IndexError: index 33 is out of bounds for axis 0 with size 1

In [21]:
# subsample of the data to ease building the model, unused in final run
eeg1_smol = eeg1[0:785000]
events1_smol = events1[0:1000]


In [22]:
# Toy data generator, unused in final run

def generate_eeg(samples, time_steps, n_features, event_types):
    # samples is Int number of trials 
    # time_steps is Int length of each trial in ms
    # n_features is Int number of EEG channels
    # event_types is Int number of stimula like lights and flashes
    signals = generate_signals(samples, time_steps, n_features)
    events = generate_events(event_types, samples)
    events_1hot = one_hot_events(events)
    return signals, events_1hot

# helper function (generate_eeg) for making EEG signal data
def generate_signals(samples, time_steps, n_features):
    # data types same as main function
    signals = np.random.random((samples, time_steps, n_features))
    return signals

# helper function (generate_eeg) for making one sample per event an
def generate_events(event_types, samples):
    # data types same as main function
    events = np.random.randint(1, event_types, samples)
    return events

In [1]:
# takes in eeg dataframe and event dataframe, cleans them, 1hot encodes the events
def clean_eeg(eeg, events, event_interval_length, eeg_slice_length):
    #event_list = []
    array_list = [] 
    index_list = []
    eeg = standardize_eeg(eeg) # function for standardizing the eeg readings
    #events_new = build_zero_events(events)
    # iterate over the rows of the events and slice out the corresponding eeg data
    for index, row in itertools.islice(events.iterrows(), event_interval_length): # loop through events data
        #build_event_list(row, event_list) #
        tmin, tmax = build_event_intervals(row, events)
        eeg_slice = cut_event_intervals(eeg, tmin, tmax)
        array_list, index_list = build_array(eeg_slice, eeg_slice_length, 
                                             index, index_list, array_list)
    y_int = events.iloc[index_list] # take the event types for the correct index
    y_int = y_int['type'].values    # take just the event types as an array
    #y_int = y_int.as_matrix()            # save the event types as a matrix
    #y, lb = one_hot_events(y_int)        # one-hot the event types and save the binarizer
    X = np.stack(array_list, axis = 0)   # stack the arrays so the whole thing is 3D
    return X, y_int                     # return the data, outputs, and the binarizer
    
        
def build_event_list(row, event_list):
    # helper function to pull event types out of event data in the right order
    event_type = getattr(row, "type")
    event_list.append(event_type)
        
def build_event_intervals(row, events):
    # helper function to get the time intervals associated with each event
    tmin = getattr(row, "latency")
    tmin_in = getattr(row, "number")
    tmax_in = tmin_in + 1
    tmax = events1.loc[tmax_in, "latency"]
    return tmin, tmax

def cut_event_intervals(eeg, tmin, tmax):
    # helper function to slice up the eeg data so each slice is associated with one event
    eeg_slice = eeg.loc[(eeg["time"] > tmin) & (eeg["time"] < tmax)]
    eeg_slice.drop(["time", "sample"], axis = 1, inplace = True)
    return eeg_slice
    
def build_array(eeg_slice, eeg_slice_length, index, index_list, array_list):
    # helper function to build an array out of the eeg slices and pad them out to a standard length
    if len(eeg_slice) < eeg_slice_length:
        index_list.append(index)
        eeg_matrix = eeg_slice.as_matrix()
        padded_matrix = np.pad(eeg_matrix, ((0, eeg_slice_length - len(eeg_matrix)), (0,0)),
                                   'constant', constant_values=0)
        array_list.append(padded_matrix)
    return array_list, index_list

def one_hot_events(events):
    # helper function for one-hot encoding the events
    events_list = list(events)
    lb = preprocessing.LabelBinarizer()
    lb.fit(events_list)
    events_1hot = lb.transform(events_list)
    return events_1hot, lb

def invert_one_hot(events, lb):
    # function for decoding one-hot, binarizer made in one_hot_events
    inv_events = lb.inverse_transform(events)
    return inv_events

In [24]:
def standardize_eeg(eeg_data):
    # breaks apart an eeg dataframe, scales the eeg readings, and reassmbles it into a dataframe
    column_list = eeg_data.columns[1:33]
    time = eeg_data['time']
    sample = eeg_data['sample']
    eeg_array = eeg_data[column_list]
    eeg_stnd = scale_data(eeg_array)
    eeg_stnd_df = pd.DataFrame(eeg_stnd, index=eeg_data.index, columns=column_list)
    eeg_stnd = pd.concat([time, eeg_stnd_df, sample], axis =1)
    return eeg_stnd

def scale_data(unscaled_data):
    # helper function for standardize_eeg, fits a scaler and transforms the data 
    scaler = StandardScaler()
    scaler.fit(unscaled_data)
    scaled_data = scaler.transform(unscaled_data)
    return scaled_data

In [25]:
# This is unused code for breaking up the "nothing happened" periods of the eeg data 
# to associate with "type 0" events. 

import math
time_steps = 1300

def build_zero_events(event_data, time_steps=time_steps):
    new_events = build_new_events(event_data, time_steps)
    events = zero_events(event_data, new_events)
    return events


def build_new_events(event_data, time_steps= time_steps):
    first_event_time = event_data['latency'].loc[1]
    number_new_intervals = math.floor(first_event_time / time_steps)
    df = pd.DataFrame(columns=['number', 'latency', 'type', 'duration'],index = range(number_new_intervals) )
    latency = 0
    for t in range(number_new_intervals):
        latency += 1300
        df.loc[t].latency = latency
        df.loc[t].type = 0
    return df

def zero_events(event_data, new_events):
    events_zeros = event_data[event_data.latency != 1]
    events_zeros= new_events.append(events_zeros)
    events_zeros = events_zeros.reset_index(drop=True)
    events_zeros['number'] = events_zeros.index + 1
    return events_zeros

### Model



In [3]:
# full dataset parameters

# define model parameters
samples = 2997  # how many trials of eeg data
n_features = 14  # how many channels of eeg in each sample
time_steps = 100 # how many ms was each sample run for
event_types = 2 #len(set(y))  # how many different event types (light, sound, etc) are there # 6 large, 4 smol

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('full_data_shuffle_24_04_2997.csv')
data.columns = ['sample_num', str(0), str(1), str(2), str(3), str(4), str(5),str(6),str(7),str(8),str(9),str(10),str(11),str(12),str(13), 'gesture']

#data = data.drop(['Unnamed: 0'], axis = 1)
d_t = pd.DataFrame(StandardScaler().fit_transform(data.drop(['sample_num', 'gesture'], axis = 1)))
d_t['gesture'] = data['gesture']
d_t['sample_num'] = data['sample_num']
data = d_t
#data = data.sample(frac=1).reset_index(drop=True)
data.shape
len(data['sample_num'].unique())

#############################

import numpy as np
import random
np_lst = []
sample_nums_rand = data['sample_num'].unique()
random.shuffle(sample_nums_rand)
for n in sample_nums_rand:
    sample = data[data['sample_num'] == n].drop(['sample_num', 'gesture'], axis = 1)
    np_lst.append(sample.values)
    
data_reshaped = np.array(np_lst).reshape((2997, 100,1,14))
np_lst

#################################

from keras.utils import to_categorical
y = data[['gesture', 'sample_num']]
l = []
for i in sample_nums_rand:
    s_y = y[y['sample_num']==i]
    if(s_y['gesture'].iloc[0] == 0):
        l.append(0)
    else:
        l.append(1)
#print(data['gesture'])      
y = to_categorical(np.array(l).reshape((len(l), )))
#y = np.array(l).reshape((len(l), ))

X =  data_reshaped

Using TensorFlow backend.


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

# use strat. shuffle split to get indices for test and training data 
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.1, random_state=32)
sss.get_n_splits(X, y)

2

In [5]:
# take the indices generated by stratified shuffle split and make the test and training datasets
for train_index, test_index in sss.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM


# code for building an LSTM with 100 neurons and dropout. Runs for 50 epochs

model = Sequential()
model.add(LSTM(100, return_sequences=False, input_shape=(time_steps, n_features)))
model.add(Dropout(0.5))
#model.add(LSTM(100)) #dramatically worse results
#model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=100, verbose = 2, validation_split = 0.1)
score = model.evaluate(X_test, y_test, batch_size=32)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ValueError: Error when checking input: expected lstm_1_input to have 3 dimensions, but got array with shape (2697, 100, 1, 14)

In [13]:
score

[0.839839494228363, 0.6100000143051147]

In [39]:
print("Accuracy: %.2f%%" % (score[1]*100))

Accuracy: 80.88%


#### saved model details
standardized

model = Sequential()
#model.add(Embedding(2, output_dim=256))
model.add(LSTM(100, input_shape=(time_steps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=16, epochs=50)
score = model.evaluate(X_test, y_test, batch_size=16)

This model run for 50 epochs had:

* binary crossentropy 0.41922928811677918

* accuracy 0.8529411764705882

In [20]:
import tensorflow as tf
#X_train,X_test,y_train,y_test = train_test_total(X, y)
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model, model_dir="/model/", 
                                        params = {'hidden_layers' : [32, 32], 'num_classes' : 4, 'learning_rate' : 0.001})
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=20,
    shuffle=True)

eeg_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

eval_train_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    shuffle=False)
eval_train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)

eval_test_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
print('Train results are:',eval_train_results)
print('Test results are:',eval_test_results)

NameError: name 'rnn_model' is not defined

In [7]:
def BCI_model(dropout = 0.5, shape = (100, 1, 14), nb_classes = 2): # shape(timestamps, 1, channels)
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(filters=25, kernel_size=(10,1), padding='same', activation='elu', input_shape=shape))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3,1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout))

    #model.add(tf.keras.layers.Conv2D(filters=50, kernel_size=(10,25), padding='same', activation=''))
    model.add(tf.keras.layers.Conv2D(filters=50, kernel_size=(10,1), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3,1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Conv2D(filters=50, kernel_size=(10,1), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3,1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense((50)))
    model.add(tf.keras.layers.Reshape((50,1)))
    model.add(tf.keras.layers.LSTM(20, dropout=0.5, recurrent_dropout=0.5, input_shape=(50,1), return_sequences=False))
    model.add(tf.keras.layers.Dense(nb_classes, activation='softmax'))
    return model
#model.summary()

In [8]:
import tensorflow as tf

model = BCI_model()
optimizer = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])
model.fit(X, y,  epochs = 50,   verbose = 1 )


Train on 2997 samples
Epoch 1/50
2997/2997 [==============================] - 12s 4ms/sample - loss: 0.6984 - acc: 0.5082
Epoch 2/50
2997/2997 [==============================] - 9s 3ms/sample - loss: 0.6977 - acc: 0.5072
Epoch 3/50
2997/2997 [==============================] - 9s 3ms/sample - loss: 0.6971 - acc: 0.4875
Epoch 4/50
2997/2997 [==============================] - 8s 3ms/sample - loss: 0.6946 - acc: 0.5068
Epoch 5/50
2997/2997 [==============================] - 8s 3ms/sample - loss: 0.6949 - acc: 0.5085
Epoch 6/50
2997/2997 [==============================] - 9s 3ms/sample - loss: 0.6941 - acc: 0.5045
Epoch 7/50
2997/2997 [==============================] - 9s 3ms/sample - loss: 0.6954 - acc: 0.4862
Epoch 8/50
2997/2997 [==============================] - 9s 3ms/sample - loss: 0.6943 - acc: 0.5038
Epoch 9/50
2997/2997 [==============================] - 9s 3ms/sample - loss: 0.6927 - acc: 0.5135
Epoch 10/50
2997/2997 [==============================] - 9s 3ms/sample - loss: 0.6954 

KeyboardInterrupt: 